In [1]:
from pyspark.sql import types as T
from pyspark.sql import functions as F 
from pyspark.sql import DataFrame
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
import sys

# from pyspark.sql.types import IntegerType
# from pyspark.sql.functions import first

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
226,application_1590480489834_0096,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Version6
#Aim: applying the new rule (i.e., changes in version5) group X should be bigger and Z smaller, comparing with version4. 
             
# a) click OR open OR registration in last 60 day
# b) click OR open OR registration in last 60 days OR soft-login OR login in last 60 days (take login dates from registration table not events from logmystst)
# c) click OR open OR registration in last 60 days OR soft-login OR login in last 60 days OR any SSA event ListingView 
    #in last 60 days OR any SSA event Search last 60 days

#===================================================================
#Version6:
#- Previous rule: open_click_60d rule for 44days: 
        #X: isactive == 1 & open_60d rule ==1: 358,690 
        #Z: isactive == 1 & open_60d rule ==0: 943,183
        #N: isactive == 0 & open_60d rule ==1: 11,353
        #M: isactive == 0 & open_60d rule ==0: 4,626,202
        
        #===================================================================================================
        #New Rule1: withColumn('Rule1_subVal_hasAnyOpenClick_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | 
                               #(F.col('registration') >=  1)) & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
        
        #X': isactive == 1 & open_60d rule ==1: 560,581 (190k increase from Previous rule) 
        #Z': isactive == 1 & open_60d rule ==0: 858,719 (85k decrease from Previous rule)
        #N': isactive == 0 & open_60d rule ==1: 27,009
        #M': isactive == 0 & open_60d rule ==0: 4,637,518
        #==================================================================================================
        ##New Rule2: withColumn('Rule2_subVal_hasAnyOpenClick_ssa_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) |
                                 #(F.col('SSA_any_60d') >=  1) | (F.col('registration') >=  1))& (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
        
        #X': isactive == 1 & open_60d rule ==1: 552,758 (195k increase from Previous rule)
        #Z': isactive == 1 & open_60d rule ==0: 714,527 (200k decrease from Previous rule)
        #N': isactive == 0 & open_60d rule ==1: 27,321
        #M': isactive == 0 & open_60d rule ==0: 4,634,992
        #===========================================================================================================
        #New Rule3: withColumn('Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) 
                              #| (F.col('login') >=  1) | (F.col('soft_login') >=  1) | (F.col('registration') >=  1))
                              # & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0))
        #X': isactive == 1 & open_60d rule ==1: 559,378 (200k increase from Previous rule)
        #Z': isactive == 1 & open_60d rule ==0: 714,527 (228k decrease from Previous rule)
        #N': isactive == 0 & open_60d rule ==1: 27,438
        #M': isactive == 0 & open_60d rule ==0: 4,634,992
        
#updates on source of data:==================================================================================================================
#ssa events 1st and 3rd to be added

#+------------------------------+------+
#|event_name                    |users |
#+------------------------------+------+
#|ListingView*                   |185858|
#|RecommenderListingsDisplayed  |171086|
#|Search*                        |94757 |
#|ListingOAFClicked            |67075 |
#|RecommenderListingClicked    |41666 |
#|SearchPaging                |22607 |
#|ListingOAFSent                |19223 |



In [2]:
#Version5.
## not logaction in ('IJM, autodeactivate', 'IJM, autoactivate')
    #excluding ('IJM, autodeactivate', 'IJM, autoactivate') from the list. Is that enough?

#===========================

# Table 1A+1B+2 
registrations_raw = spark.sql("""
    select 
        UID.id as user_id
        , to_date(dcreated) as reg_date
        , passwordset
        , subscriptionid
        , to_date(subscriptiondate) as sub_date
        , isactive
        , active as AccountValidation
        , manualunsubscribe
        , to_date(datelastsoftlogin) as last_softlog
        , to_date(dlastlogin) as last_login
        , to_date(logdatetime) as log_date
        , logaction
        , count(logdatetime) as n_events
    from stepstonede.userlogin UID
    left join stepstonede.subscription SUB
    on UID.id = SUB.userlogin_id
    left join stepstonede.logmystst LOG
    on 
        UID.id = LOG.userloginid 
        and not logaction in ('IJM, autodeactivate', 'IJM, autoactivate')
    where
        dcreated >= '2018-01-01'
    group by
        UID.id 
        , reg_date
        , passwordset
        , subscriptionid
        , sub_date
        , isactive
        , active
        , manualunsubscribe
        , last_softlog
        , last_login
        , log_date
        , logaction
    order by reg_date, log_date""")
#  or subscriptiondate >= '2020-04-01'



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Version6 update:
# last_login", "last_softlog", "log_date" 
    
#Version4B update:
# Create registration dataframe by AccountValidation
registration_df = registrations_raw.select("user_id", "reg_date", "AccountValidation", "last_login", "last_softlog", "log_date").distinct().\
    withColumn("considered_date", F.to_date(F.lit(registrations_raw.select(F.max(F.col("reg_date")).alias("max_date")).collect()[0][0])))
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#check registration_df for one user (user_id == 8482301)
#registration_df.filter(F.col("user_id") == "8482301").show()

#Not:
#1. Registration event in last 44 days: 
    # considered_date is much after registration date, because considered_date == Maximum of reg_date. so, for anyone who is considered on 
    # a given day (e.g., 2020-05-26), we definitely have them registered at some point in the past. 
    # So, for any rule we define, users are considered have been definitely registered. So, the registration within 1-60 days is true in the new rule. 
    
#2. login and soft login events in last 44 days:
    # accordingly, based on the table bellow, the login_state and soft_login_state are defined as:
    #withColumn("login_state", F.when(F.col("last_login") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0)))
    #withColumn("soft_login_state", F.when(F.col("last_softlog") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))) 
    
# +-------+----------+-----------------+----------+------------+----------+---------------+
# |user_id|  reg_date|AccountValidation|last_login|last_softlog|  log_date|considered_date|
# +-------+----------+-----------------+----------+------------+----------+---------------+
# |8482301|2019-10-31|             true|2020-05-23|  2020-05-14|2019-10-31|     2020-05-26|
# |8482301|2019-10-31|             true|2020-05-23|  2020-05-14|2019-11-01|     2020-05-26|
# |8482301|2019-10-31|             true|2020-05-23|  2020-05-14|2019-11-07|     2020-05-26|
# |8482301|2019-10-31|             true|2020-05-23|  2020-05-14|2019-11-08|     2020-05-26|


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Version6 update: 
# create login_state and soft_login_state based on "last_login", "last_softlog", "log_date"
    #withColumn("login_state", F.when(F.col("last_login") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0)))
    #withColumn("soft_login_state", F.when(F.col("last_softlog") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))) 

#=========================================================================================
#Version4B update: withColumn("validated_account", F.when(F.col("AccountValidation") == True , F.lit(1)).otherwise(F.lit(0))).
# add IJM subscription date to registration data frame and remove unsubscribed users from registation data frame

IJM_subscription_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, subscription"])).groupBy("user_id").agg(F.min(F.col("log_date")).alias("sub_date"))
    
registration_df = registration_df.join(IJM_subscription_df, "user_id", "left")

IJM_unsubscription_df = registrations_raw. \
    filter(F.col("manualunsubscribe") == True). \
    select("user_id", "manualunsubscribe").distinct()

registration_df = registration_df.join(IJM_unsubscription_df, "user_id", "left_anti").\
    withColumn("validated_account", F.when(F.col("AccountValidation") == True , F.lit(1)).otherwise(F.lit(0))).\
    withColumn("sub_state", F.when(F.col("sub_date") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))).\
    withColumn("login_state", F.when(F.col("last_login") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))).\
    withColumn("soft_login_state", F.when(F.col("last_softlog") <= F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))).\
    select("user_id", "reg_date", "validated_account", "sub_state", "considered_date", "sub_date", "log_date", "login_state", "soft_login_state")

# registration_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#check registration_df for one user (user_id == 8482301)
#registration_df.filter(F.col("user_id") == "8482301").show()

# +-------+----------+-----------------+---------+---------------+----------+----------+-----------+----------------+
# |user_id|  reg_date|validated_account|sub_state|considered_date|  sub_date|  log_date|login_state|soft_login_state|
# +-------+----------+-----------------+---------+---------------+----------+----------+-----------+----------------+
# |8482301|2019-10-31|                1|        1|     2020-05-26|2019-10-31|2020-03-22|          1|               1|
# |8482301|2019-10-31|                1|        1|     2020-05-26|2019-10-31|2020-03-24|          1|               1|
# |8482301|2019-10-31|                1|        1|     2020-05-26|2019-10-31|2020-03-30|          1|               1|
# |8482301|2019-10-31|                1|        1|     2020-05-26|2019-10-31|2020-04-02|          1|               1|
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#Table3: Website_visits_df
Website_visits_df = spark.sql("""
    select distinct user_id, user_hash, job_agent_id, to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_de
    where exclude_hit <= 0 and year = 2020

    union all

    select distinct user_id, user_hash, job_agent_id, to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_android_global AA
    where AA.job_site like '%-de' and exclude_hit <= 0 and year = 2020

    union all

    select distinct user_id, user_hash, job_agent_id, to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_iphoneapp_global AA
    where AA.job_site like '%-de' and exclude_hit <= 0 and year = 2020""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
#E. how many website visit per day? about 100k (Max number of distinct users with website visit: 109,223)

#Website_visits_df = Website_visits_df.groupBy('visit_date').agg(F.countDistinct(F.col("user_id")).alias("userCount_visits_Perday")).sort(desc("userCount_visits_Perday")).show()

#+----------+-----------------------+
#|visit_date|userCount_visits_Perday|
#+----------+-----------------------+
#|2020-05-19|                 109223|
#|2020-04-28|                 108139|
#|2020-05-12|                 106296|
#|2020-05-06|                 103219|
#|2020-05-13|                 102728|
#|2020-05-11|                 101208|
#|2020-03-11|                 100194|
#|2020-05-04|                  99195|
#|2020-05-18|                  97960|
#|2020-05-05|                  97763|
#|2020-05-14|                  96560|
#|2020-04-29|                  95653|
#|2020-04-20|                  94894|
#|2020-04-21|                  93818|
#|2020-05-07|                  93008|
#|2020-03-12|                  92912|
#|2020-04-27|                  92317|
#|2020-05-20|                  90976|
#|2020-05-15|                  90356|
#|2020-04-30|                  90220|
#+----------+-----------------------+
                                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#Jobagent_id: match jobagent_id from stepstonede.userlogin_jobuser to user_id in Website_visits_df

jobuser_original_df = spark.sql("""
    select distinct userlogin_id, jobuser_id as job_agent_id
    from stepstonede.userlogin_jobuser
    """)
Website_visits_jobuser_df = Website_visits_df.join(jobuser_original_df, "job_agent_id", "left")

#If there is only a user_hash_id then match it to a user_id only if that user_hash_id is used with only one user_id
user_hash_df = spark.sql("""
    with hash_count as (
        select user_hash, count(distinct user_id) as n_UID
        from presentation.adobe_user_id_hash_de
        group by user_hash)
    select A.user_hash, A.user_id as user_id_hash
    from presentation.adobe_user_id_hash_de A
    right join hash_count B 
    on 
        A.user_hash = B.user_hash 
        and B.n_UID = 1""")
Website_visits_df = Website_visits_jobuser_df.join(user_hash_df, "user_hash", "left")

#from Website_visits_df_final map user_ids from user_hash_df and jobuser_df into Website_visits_df to fill null values
Website_visits_df = Website_visits_df.select("user_id", "userlogin_id", "user_id_hash", "visit_date")


Website_visits_df = Website_visits_df.withColumn('new_user_id', 
                                                               F.when(F.col('user_id').isNotNull(), col('user_id')). \
                                                               otherwise(F.when(F.col('userlogin_id').isNotNull(), col('userlogin_id'))))

Website_visits_df = Website_visits_df.withColumn('new_user_id_1', 
                                                               F.when(F.col('new_user_id').isNotNull(), col('new_user_id')). \
                                                               otherwise(F.when(F.col('user_id_hash').isNotNull(), col('user_id_hash'))))

Website_visits_df = Website_visits_df.filter(F.col("new_user_id_1").isNotNull()).select(F.col("new_user_id_1").alias("user_id"), "visit_date")

#Bring website visit (without null user_id) and email data frames together
#web_email_df: Join Final_Website_visits_df table with email_log_df
simulation_visits_df = registration_df.join(Website_visits_df, "user_id", "left")
simulation_visits_df = simulation_visits_df.select("user_id", "reg_date", "validated_account", "considered_date","sub_state", "visit_date", 
                                                   "sub_date", "log_date", "login_state", "soft_login_state")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#updates: 
#3. change date range from 0-60 to 1-60
#2. Add date_difference_sub
#1. Add considered rule based on website visits:

simulation_visits_df = simulation_visits_df.\
    withColumn('date_difference', datediff('considered_date', 'visit_date')).\
    groupBy("user_id", "reg_date", "validated_account", "considered_date", "sub_state", "sub_date", "log_date", "login_state", "soft_login_state"). \
    agg(F.sum(F.when((F.col("date_difference") <= 60) & (F.col("date_difference") >= 1), F.lit(1)).otherwise(0)).alias("visits_60d")). \
    withColumn('hasVisit_60d', F.when(F.col('visits_60d') >=  1, 1).otherwise(0)).\
    withColumn('subVal_hasVisit_60d', F.when((F.col('visits_60d') >=  1) & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0))

# simulation_visits_df.show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
#check_visits_rule.crosstab("SubVal_hasVisit", "isactive").show()

#Initaial finding:
# There are about K900 (937867) active user_id in subscription tables from 2020-01-01 (5 months and 2 weeks) but only about 20% (208643) recieve emails that accounts for about 80% (643673) of users out of reach.
#Question: What is the reasons behind this missmatch?
#+------------------------+------+------+
#|SubVal_hasVisit_isactive|  null|  true|
#+------------------------+------+------+
#|                       1|  5079|208643|
#|                       0|541613|643673|
#|                    null|     0| 85551|
#+------------------------+------+------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Observations: 
# many with active subscriptions do not seem to have website visits
# small number of users with inactive subscriptions have website visits
# quite a few users with active subscriptions are not contained in our dataset
# next try SSA data for activitiy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Version6.
#Add 1st and 3rd ssa events only, because they are the most representative events for audiance rule. why? not exactly clear to me. 


#+------------------------------+------+
#|event_name                    |users |
#+------------------------------+------+
#|ListingView*                   |185858|
#|RecommenderListingsDisplayed*  |171086|
#|Search*                        |94757 |
#|ListingOAFClicked             |67075 |


#==================================
#Note: add tracking_id that contains the same user id (same integer), but it is for not logged in identified users 


ssa_log_df = spark.sql("""
    select distinct
        to_date(log_date) as ssa_date
        , event_name
        , tracking_my_stepstone_id as user_id
        , case when event_name = 'ListingView' then 1 else 0 end as listing_view
        , 1 as any_ssa
    from presentation.ssa_fact 
    Where year >= 2020 and month >= 3 and country = 'DE' and 
          event_name in ('ListingView', 
                         'Search') """)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#email data before the rules
selligent_df = spark.sql("""
    SELECT
        to_date(concat_ws('-', cast(year as string), cast(month as string), cast(day as string))) as action_day,
        user_login_id as user_id, 
        floor(sum(sent)) as send, 
        floor(sum(opened)) as total_open,
        count(distinct case when opened > 0 then ditem_id end) as unique_open,
        floor(sum(clicked)) as total_clicks,
        count(distinct case when clicked > 0 then ditem_id end) as unique_clicks
    FROM live_raw_selligent_de_stepstone.flags_enriched fe
    GROUP BY action_day, user_id""").\
    union(spark.sql("""
    select
        to_date(concat_ws('-', cast(year as string), cast(month as string), cast(day as string))) as action_day,
        userlogin_id as user_id, 
        0 as send,
        0 as total_open,
        0 as unique_open,
        count(*) as total_clicks,
        count(distinct jaid) as unique_clicks
    from stepstonede.jaclickofferid A
    right join stepstonede.userlogin_jobuser B on A.jaid = B.jobuser_id
    group by action_day, user_id
    """))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
selligent_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
#Version6:
#Question: Where in this funnel out new audience rule is located?
    #when a user is considered (which is the last day a user registered)
    #So:
    # A: size of possible considered audiance for a given date is total of disctinct users on each considered date
    # B: size of cosidered audiance within each rule is total of distinct users that are considered in that rule.
    #Note=============================================================================================
    # A is always bigger than B, because we send email to only to specific users with specific interactions (e.g., those who opened or clicked on previouse emails in 44 days ago) with the system.

#checking:
    #before the rule:
    #how many users-send email per day? Max=4,179,658 (between 2M to 4M)
    #How many "opens" per day? Max 241 per day
    #How many "clicks" per day? Max 307 per day
    #How many "send" per day? Max 331
    
#=================================================================================
#before the rule:
#how many users-send email per day? between 2M to 4M Max=4,179,658 to 1,684,718

# selligent_df.groupBy("action_day").agg(F.countDistinct(F.col("user_id")).alias("user_perday")).sort(desc("user_perday")).show()

# +----------+-----------+
# |action_day|user_perday|
# +----------+-----------+
# |2019-07-15|    4179658|
# |      null|    3533942|


#======================================

#How many "opens" per day? Max 241 per day

# selligent_df.groupBy("action_day").agg(F.countDistinct(F.col("total_open")).alias("open_perday")).sort(desc("open_perday")).show()

# +----------+-----------+
# |action_day|open_perday|
# +----------+-----------+
# |2018-06-18|        241|
# |2018-06-19|        239|
# |2018-06-25|        237|

# +----------+-----------+

#==========================================

#How many "clicks" per day? Max 307 per day

# selligent_df.groupBy("action_day").agg(F.countDistinct(F.col("total_clicks")).alias("click_perday")).sort(desc("click_perday")).show()

# +----------+------------+
# |action_day|click_perday|
# +----------+------------+
# |      null|         307|
# |2018-12-21|         191|
# |2018-12-20|         155|
# 

#======================================================
#How many "send" per day? Max 331
# selligent_df.groupBy("action_day").agg(F.countDistinct(F.col("send")).alias("send_perday")).sort(desc("send_perday")).show()
# +----------+-----------+
# |action_day|send_perday|
# +----------+-----------+
# |2019-03-25|        331|
# |2019-03-26|        309|
# |2019-03-28|        308|
# |2019-03-27|        300|
# |2019-03-24|        294|
# |
# +----------+-----------+


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#1.1. left join selligent_df with simulation_visits_df
simulation_visits_selligent_df = selligent_df.join(simulation_visits_df, "user_id", "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# simulation_visits_selligent_df.filter(F.col("user_id") == 8482301).show()

# user_id|action_day|send|total_open|unique_open|total_clicks|unique_clicks|  reg_date|validated_account|considered_date|sub_state|  sub_date|  log_date|login_state|soft_login_state|visits_60d|hasVisit_60d|subVal_hasVisit_60d|
# +-------+----------+----+----------+-----------+------------+-------------+----------+-----------------+---------------+---------+----------+----------+-----------+----------------+----------+------------+-------------------+
# |8482301|2020-04-20|   6|         7|          7|           1|            1|2019-10-31|                1|     2020-05-26|        1|2019-10-31|2019-10-31|          1|               1|        50|           1|                  1|

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#Version6:

# a) click OR open OR registration in last 60 days
# b) click OR open OR registration in last 60 days OR soft-login OR login in last 60 days (take login dates from registration table not events from logmystst)
# c) click OR open OR registration in last 60 days OR soft-login OR login in last 60 days OR any SSA event ListingView 
    #in last 60 days OR any SSA event Search last 60 days
#groupby 
    # user_id|action_day|send|total_open|unique_open|total_clicks|unique_clicks|  
    #reg_date|validated_account|considered_date|sub_state|  sub_date|  
    #log_date|login_state|soft_login_state|visits_60d|hasVisit_60d|subVal_hasVisit_60d|

simulation_visits_selligent_ssa_df = simulation_visits_selligent_df.join(ssa_log_df, "user_id", "left").\
    withColumn('date_difference_reg', datediff('considered_date', 'reg_date')). \
    withColumn('date_difference_sub', datediff('considered_date', 'sub_date')). \
    withColumn('date_difference_ssa', datediff('considered_date', 'ssa_date')).\
    withColumn('date_difference_log', datediff('considered_date', 'log_date')).\
    withColumn('date_difference_actionDay', datediff('considered_date', 'action_day')). \
    groupBy("user_id", "action_day", "reg_date", "validated_account", "considered_date", "sub_state", "log_date", "login_state", "soft_login_state", "hasVisit_60d", "subVal_hasVisit_60d"). \
    agg(F.sum(F.when((F.col("date_difference_ssa") <= 60) & (F.col("date_difference_ssa") >= 1) & (F.col('any_ssa') > 0), F.lit(1)).otherwise(0)).alias("SSA_any_60d"),
        F.sum(F.when((F.col("date_difference_log") <= 60) & (F.col("date_difference_log") >= 1) & (F.col('login_state') > 0), F.lit(1)).otherwise(0)).alias("login"),
        F.sum(F.when((F.col("date_difference_log") <= 60) & (F.col("date_difference_log") >= 1) & (F.col('soft_login_state') > 0), F.lit(1)).otherwise(0)).alias("soft_login"),
        F.sum(F.when((F.col("date_difference_actionDay") <= 60) & (F.col("date_difference_actionDay") >= 1) & (F.col('total_open') > 0), F.lit(1)).otherwise(0)).alias("open"),
        F.sum(F.when((F.col("date_difference_actionDay") <= 60) & (F.col("date_difference_actionDay") >= 1) & (F.col('total_clicks') > 0), F.lit(1)).otherwise(0)).alias("click"),
        F.sum(F.when((F.col("date_difference_reg") <= 60) & (F.col("date_difference_reg") >= 1), F.lit(1)).otherwise(0)).alias("registration")).\
    withColumn('subVal_hasAnySSA_reg_60d', F.when(((F.col('SSA_any_60d') >=  1) | (F.col('registration') >=  1)) & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
    withColumn('subVal_hasAnyLogin_reg_60d', F.when(((F.col('login_state') >=  1) | (F.col('registration') >=  1))& (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
    withColumn('subVal_hasAnySoftLogin_reg_60d', F.when(((F.col('soft_login_state') >=  1) | (F.col('registration') >=  1))& (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
    withColumn('subVal_hasAnyLogin_SoftLogin_reg_60d', F.when(((F.col('login') >=  1) | (F.col('soft_login') >=  1) | (F.col('registration') >=  1))
                                                          & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
    withColumn('Rule1_subVal_hasAnyOpenClick_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('registration') >=  1))
                                                          & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
    withColumn('Rule2_subVal_hasAnyOpenClick_ssa_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) | (F.col('registration') >=  1))
                                                          & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
    withColumn('Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) 
                                                               | (F.col('login') >=  1) | (F.col('soft_login') >=  1) | (F.col('registration') >=  1))
                                                               & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# check SSA simulation result against state of subscription table
subs_df = spark.sql("""
    select userlogin_id as user_id, subscriptiondate, isactive, manualunsubscribe
    from stepstonede.subscription
    where isactive == 1 and 
    (manualunsubscribe is null or not manualunsubscribe)
    and subscriptiondate >= '2018-01-01' 
    """)


#and subscriptiondate >= '2018-01-01'
# roughly half of users cannot be mapped > why is that? missmatch in user_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#left join seligent+simulation table with subscription table 
simulation_visits_selligent_ssa_sub_df = simulation_visits_selligent_ssa_df.join(subs_df, "user_id", "left")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Version5:(range date has increased to 60days + "jaclockofferid.jaid = userlogin_jobuser.id")
#- Previouse rule: open_click_60d rule for 60days: 
        #X: isactive == 1 & open_60d rule ==1: 358,690 
        #Z: isactive == 1 & open_60d rule ==0: 943,183
        #N: isactive == 0 & open_60d rule ==1: 11,353
        #M: isactive == 0 & open_60d rule ==0: 4,626,202
        

#===================================================================================================
#New Rule1: withColumn('Rule1_subVal_hasAnyOpenClick_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('registration') >=  1))
#                                                         & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
        #X': isactive == 1 & open_60d rule ==1: 538,275 (150k increase) 
        #Z': isactive == 1 & open_60d rule ==0: 846655 (100k decrease)
        #N': isactive == 0 & open_60d rule ==1: 38704
        #M': isactive == 0 & open_60d rule ==0: 4655601

    

simulation_visits_selligent_ssa_sub_df.groupBy('isactive', 'Rule1_subVal_hasAnyOpenClick_reg_60d').agg(F.countDistinct(F.col("user_id"))).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------------------+-----------------------+
|isactive|Rule1_subVal_hasAnyOpenClick_reg_60d|count(DISTINCT user_id)|
+--------+------------------------------------+-----------------------+
|    true|                                   0|                 846655|
|    null|                                   1|                  38704|
|    true|                                   1|                 538275|
|    null|                                   0|                4655601|
+--------+------------------------------------+-----------------------+

In [17]:
#Version5:(range date has increased to 60days + "jaclockofferid.jaid = userlogin_jobuser.id")
#- Previouse rule: open_click_60d rule for 44days: 
        #X: isactive == 1 & open_60d rule ==1: 358,690 
        #Z: isactive == 1 & open_60d rule ==0: 943,183
        #N: isactive == 0 & open_60d rule ==1: 11,353
        #M: isactive == 0 & open_60d rule ==0: 4,626,202
        

#===================================================================================================
#44days
#New Rule2: withColumn('Rule2_subVal_hasAnyOpenClick_ssa_reg_45d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) | (F.col('registration') >=  1))
#                                                           & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
        #X': isactive == 1 & open_60d rule ==1: 552,758
        #Z': isactive == 1 & open_60d rule ==0: 714,527
        #N': isactive == 0 & open_60d rule ==1: 27,321
        #M': isactive == 0 & open_60d rule ==0: 4,634,992
        
#===========================================================
#59 days
#New Rule2: withColumn('Rule2_subVal_hasAnyOpenClick_ssa_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) | (F.col('registration') >=  1))
#                                                           & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
        #X': isactive == 1 & open_60d rule ==1: 579,904
        #Z': isactive == 1 & open_60d rule ==0: 683,784
        #N': isactive == 0 & open_60d rule ==1: 30372
        #M': isactive == 0 & open_60d rule ==0: 4636545
    
simulation_visits_selligent_ssa_sub_df.groupBy('isactive', 'Rule2_subVal_hasAnyOpenClick_ssa_reg_60d').agg(F.countDistinct(F.col("user_id"))).show()

#conclusion:
#For 20days increase in data time we have about 50k improvement in groupZ

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------------------------------+-----------------------+
|isactive|Rule2_subVal_hasAnyOpenClick_ssa_reg_60d|count(DISTINCT user_id)|
+--------+----------------------------------------+-----------------------+
|    true|                                       0|                 677537|
|    null|                                       1|                  39017|
|    true|                                       1|                 544784|
|    null|                                       0|                4652235|
+--------+----------------------------------------+-----------------------+

In [18]:
#Version5:(range date has increased to 60days + "jaclockofferid.jaid = userlogin_jobuser.id")
#- Previouse rule: open_click_60d rule for 44days: 
        #X: isactive == 1 & open_60d rule ==1: 358,690 
        #Z: isactive == 1 & open_60d rule ==0: 943,183
        #N: isactive == 0 & open_60d rule ==1: 11,353
        #M: isactive == 0 & open_60d rule ==0: 4,626,202
        

#===================================================================================================
#New Rule3: withColumn('Rule3_subVal_hasAnyOpenClick_ssa_login_reg_45d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) 
#                                                                | (F.col('login') >=  1) | (F.col('soft_login') >=  1) | (F.col('registration') >=  1))
#                                                                & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0))
        #X': isactive == 1 & open_60d rule ==1: 559,378
        #Z': isactive == 1 & open_60d rule ==0: 714,527
        #N': isactive == 0 & open_60d rule ==1: 27,438
        #M': isactive == 0 & open_60d rule ==0: 4,634,992
        
#===========================================================
#59 days
#New Rule2: withColumn('Rule2_subVal_hasAnyOpenClick_ssa_reg_60d', F.when(((F.col('open') >=  1) | (F.col('click') >=  1) | (F.col('SSA_any_60d') >=  1) | (F.col('registration') >=  1))
#                                                           & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0)).\
        #X': isactive == 1 & open_60d rule ==1: 
        #Z': isactive == 1 & open_60d rule ==0: 
        #N': isactive == 0 & open_60d rule ==1: 
        #M': isactive == 0 & open_60d rule ==0: 
    

    
simulation_visits_selligent_ssa_sub_df.groupBy('isactive', 'Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d').agg(F.countDistinct(F.col("user_id"))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------------------------------------+-----------------------+
|isactive|Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d|count(DISTINCT user_id)|
+--------+----------------------------------------------+-----------------------+
|    true|                                             0|                 677549|
|    null|                                             1|                  39229|
|    true|                                             1|                 553086|
|    null|                                             0|                4652427|
+--------+----------------------------------------------+-----------------------+

In [19]:
#Updates from this cell to out code: using validation account in registration table  

# Q: how important are validate_account and sub_state? is there any error in our logic?
# here we see we have to do some cleaning up of "validated account" and "sub_state"
simulation_visits_selligent_ssa_sub_df.groupBy('isactive', 'validated_account', "sub_state").agg(F.countDistinct(F.col("user_id"))).show()

#Version5
# +--------+-----------------+---------+-----------------------+
# |isactive|validated_account|sub_state|count(DISTINCT user_id)|
# +--------+-----------------+---------+-----------------------+
# |    null|                1|        0|                 301523|*** :) incread
# |    null|             null|     null|                4102505|
# |    true|             null|     null|                  82585|* Not changed! :(
# |    true|                0|        1|                  13187|** :)
# |    null|                1|        1|                  68160|*increased :)
# |    true|                1|        1|                 844106|
# |    null|                0|        0|                 164307|
# |    true|                0|        0|                     60|
# |    true|                1|        0|                   3558|
# |    null|                0|        1|                    859|
# +--------+-----------------+---------+-----------------------+


#============================================================================

#version4:
#+--------+-----------------+---------+-----------------------+
#|isactive|validated_account|sub_state|count(DISTINCT user_id)|
#+--------+-----------------+---------+-----------------------+
#|    null|                1|        0|                 258236| TO LOOK AT LATER: that may be an opportunity - to look at later
#|    null|             null|     null|                4098988| ALEX_find out: who are these users? why to they appear in our dataset?
#|    true|             null|     null|                  82726| IMPORTANT: why aren't thos people in our dataset? 
#|    true|                0|        1|                  62429| SOLVED: is our "validated_account" logic ok? include "active" field from userlogin table in registration df and compare
#|    null|                1|        1|                  61876| TO LOOK AT LATER: this is an opportunity - no need to look into further
#|    true|                1|        1|                 796026| OK
#|    null|                0|        0|                 206784| OK
#|    true|                0|        0|                   3433| OK - too small
#|    true|                1|        0|                    260| OK - too small
#|    null|                0|        1|                    363| OK - too small
#+--------+-----------------+---------+-----------------------+


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+---------+-----------------------+
|isactive|validated_account|sub_state|count(DISTINCT user_id)|
+--------+-----------------+---------+-----------------------+
|    null|                1|        0|                 301725|
|    null|             null|     null|                4113904|
|    true|             null|     null|                  82974|
|    true|                0|        1|                  12119|
|    null|                1|        1|                  80982|
|    true|                1|        1|                 855066|
|    null|                0|        0|                 166213|
|    true|                0|        0|                     43|
|    true|                1|        0|                   3495|
|    null|                0|        1|                   1783|
+--------+-----------------+---------+-----------------------+

In [ ]:
#version5: (range date has increased to 60days + "jaclockofferid.jaid = userlogin_jobuser.id")
#Activities of GroupZ in 60 days
    #total number of sends 
    #total number of clicks 
    #total number of opens 
    #total number of registrations 
    #total number of login  
    #total number of ssa actions 


In [20]:
#Activities of groupZ against Rule1_subVal_hasAnyOpenClick_reg_60d
simulation_visits_selligent_ssa_sub_GroupZ_df = simulation_visits_selligent_ssa_sub_df.filter((subs_df.isactive ==1) & 
                                                                                              (simulation_visits_selligent_ssa_df.Rule1_subVal_hasAnyOpenClick_reg_60d == 0)).\
                                                                                              groupBy('isactive').\
                                                                                              agg(F.sum(F.countDistinct(F.col("send"))).alias("total_send_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("total_open"))).alias("total_open_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("total_clicks"))).alias("total_click_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("registration"))).alias("total_registration_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("login"))).alias("total_Login_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("soft_login"))).alias("total_SoftLogin_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("SSA_any_60d"))).alias("total_ssa_perUser"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`send`' given input columns: [subVal_hasAnyLogin_SoftLogin_reg_60d, soft_login, reg_date, open, stepstonede.subscription.subscriptiondate, subVal_hasAnySSA_reg_60d, user_id, considered_date, login_state, registration, Rule2_subVal_hasAnyOpenClick_ssa_reg_60d, hasVisit_60d, log_date, stepstonede.subscription.manualunsubscribe, sub_state, soft_login_state, action_day, Rule1_subVal_hasAnyOpenClick_reg_60d, SSA_any_60d, subVal_hasVisit_60d, subVal_hasAnyLogin_reg_60d, click, login, validated_account, Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d, stepstonede.subscription.isactive, subVal_hasAnySoftLogin_reg_60d];;\n'Aggregate [isactive#2967], [isactive#2967, sum(count(distinct 'send)) AS total_send_perUser#3415, sum(count(distinct 'total_open)) AS total_open_perUser#3418, sum(count(distinct 'total_clicks)) AS total_click_perUser#3421, sum(count(distinct registration#2792L)) AS total_registration_perUser#3424L, sum(count(distinct login#2784L)) AS total_Login_perUser#3427L,

In [21]:
#Activities of groupZ against Rule2_subVal_hasAnyOpenClick_ssa_reg_60d
simulation_visits_selligent_ssa_sub_GroupZ_df = simulation_visits_selligent_ssa_sub_df.filter((subs_df.isactive ==1) & 
                                                                                              (simulation_visits_selligent_ssa_df.Rule2_subVal_hasAnyOpenClick_ssa_reg_60d == 0)).\
                                                                                              groupBy('isactive').\
                                                                                              agg(F.sum(F.countDistinct(F.col("send"))).alias("total_send_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("total_open"))).alias("total_open_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("total_clicks"))).alias("total_click_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("registration"))).alias("total_registration_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("login"))).alias("total_Login_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("soft_login"))).alias("total_SoftLogin_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("SSA_any_60d"))).alias("total_ssa_perUser"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`send`' given input columns: [subVal_hasAnyLogin_SoftLogin_reg_60d, soft_login, reg_date, open, stepstonede.subscription.subscriptiondate, subVal_hasAnySSA_reg_60d, user_id, considered_date, login_state, registration, Rule2_subVal_hasAnyOpenClick_ssa_reg_60d, hasVisit_60d, log_date, stepstonede.subscription.manualunsubscribe, sub_state, soft_login_state, action_day, Rule1_subVal_hasAnyOpenClick_reg_60d, SSA_any_60d, subVal_hasVisit_60d, subVal_hasAnyLogin_reg_60d, click, login, validated_account, Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d, stepstonede.subscription.isactive, subVal_hasAnySoftLogin_reg_60d];;\n'Aggregate [isactive#2967], [isactive#2967, sum(count(distinct 'send)) AS total_send_perUser#3463, sum(count(distinct 'total_open)) AS total_open_perUser#3466, sum(count(distinct 'total_clicks)) AS total_click_perUser#3469, sum(count(distinct registration#2792L)) AS total_registration_perUser#3472L, sum(count(distinct login#2784L)) AS total_Login_perUser#3475L,

In [22]:
#Activities of groupZ against Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d
simulation_visits_selligent_ssa_sub_GroupZ_df = simulation_visits_selligent_ssa_sub_df.filter((subs_df.isactive ==1) & 
                                                                                              (simulation_visits_selligent_ssa_df.Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d == 0)).\
                                                                                              groupBy('isactive').\
                                                                                              agg(F.sum(F.countDistinct(F.col("send"))).alias("total_send_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("total_open"))).alias("total_open_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("total_clicks"))).alias("total_click_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("registration"))).alias("total_registration_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("login"))).alias("total_Login_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("soft_login"))).alias("total_SoftLogin_perUser"),
                                                                                                  F.sum(F.countDistinct(F.col("SSA_any_60d"))).alias("total_ssa_perUser"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`send`' given input columns: [subVal_hasAnyLogin_SoftLogin_reg_60d, soft_login, reg_date, open, stepstonede.subscription.subscriptiondate, subVal_hasAnySSA_reg_60d, user_id, considered_date, login_state, registration, Rule2_subVal_hasAnyOpenClick_ssa_reg_60d, hasVisit_60d, log_date, stepstonede.subscription.manualunsubscribe, sub_state, soft_login_state, action_day, Rule1_subVal_hasAnyOpenClick_reg_60d, SSA_any_60d, subVal_hasVisit_60d, subVal_hasAnyLogin_reg_60d, click, login, validated_account, Rule3_subVal_hasAnyOpenClick_ssa_login_reg_60d, stepstonede.subscription.isactive, subVal_hasAnySoftLogin_reg_60d];;\n'Aggregate [isactive#2967], [isactive#2967, sum(count(distinct 'send)) AS total_send_perUser#3511, sum(count(distinct 'total_open)) AS total_open_perUser#3514, sum(count(distinct 'total_clicks)) AS total_click_perUser#3517, sum(count(distinct registration#2792L)) AS total_registration_perUser#3520L, sum(count(distinct login#2784L)) AS total_Login_perUser#3523L,